In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import pandas as pd
from typing import Any
import re  
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import asyncio
import nest_asyncio

In [ ]:
class PandasCodeGenerator:
    def __init__(self):
        model_id = "mistralai/Mistral-7B-Instruct-v0.3"
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            torch_dtype=torch.float16,
            quantization_config=None,  # Colab Pro: 4-bit با bitsandbytes
            # برای Colab رایگان: حذف quantization و استفاده از CPU یا T4
        )
        self.pipe = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_new_tokens=100,
            temperature=0.1,
            do_sample=False
        )

    def generate_pandas_code(self, question: str, df_info: str) -> str:
      prompt = f"""You are a Pandas expert. Your task is to generate ONLY a valid Pandas expression using `df`.

  DataFrame info:
  {df_info}

  Sample data:
  - Sex: 'Male', 'Female'
  - Educations: 'Diploma', 'Elementary', 'HighSchool'
  - isactive: 0 or 1
  - Age: numbers like 15, 34, 41
  - Marriage: 'Married', 'Single'
  - Position: 'Ring', 'Security', 'Flyer', etc.

  RULES:
  1. Use EXACT column names: 'Sex', 'Educations', 'isactive', etc.
  2. For gender: 'Male' or 'Female' (English only)
  3. For education: 'Diploma', 'Elementary', 'HighSchool'
  4. Return ONLY the Pandas code. NO Persian, NO explanation, NO markdown.
  5. If result is a number: use .mean(), .count(), .sum()
  6. If result is count: use .shape[0]
  7. If result is list: use .unique() or .value_counts()

  EXAMPLES:
  Q: چند نفر دیپلم دارند؟
  A: df[df['Educations'] == 'Diploma'].shape[0]

  Q: میانگین نرخ نگهداشت زنان
  A: df[df['Sex'] == 'Female']['gender_retention_rate'].mean()

  Q: چند نفر متاهل و فعال هستند؟
  A: df[(df['Marriage'] == 'Married') & (df['isactive'] == 1)].shape[0]

  Q: شایع‌ترین موقعیت شغلی
  A: df['Position'].value_counts().idxmax()

  User question: {question}

  Answer with ONLY the Pandas code:
  """

      # تولید متن
      result = self.pipe(prompt)[0]['generated_text']

      # استخراج بعد از آخرین "Answer with ONLY the Pandas code:"
      if "Answer with ONLY the Pandas code:" in result:
          code = result.split("Answer with ONLY the Pandas code:")[-1].strip()
      else:
          code = result.strip()

      # تمیز کردن خروجی
      lines = [line.strip() for line in code.split("\n") if line.strip()]
      code = lines[0] if lines else ""

      # حذف ```python و ```
      if code.startswith("```"):
          code = code.split("```", 1)[-1].split("```", 1)[0].strip()

      # حذف هر چیزی غیر از کد پانداس
      import re
      code = re.sub(r'^[^(df].*', '', code)  # فقط خطی که با df شروع بشه
      code = re.sub(r'[^\w\.\[\]\'"=><&|() ]', '', code)  # فقط کاراکترهای مجاز

      # اگر خالی بود، یه کد پیش‌فرض بده
      if not code or not code.startswith("df"):
          code = "df.shape[0]"  # fallback

      return code

In [ ]:
# executor

class SafeExecutor:
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.allowed_globals = {
            'df': self.df,
            'pd': pd,
            '__builtins__': {}
        }

    def execute(self, code: str) -> Any:
      # فقط کدهایی که با df شروع بشن
      if not code.strip().startswith("df"):
          return "خطا: کد باید با df شروع شود"

      forbidden = ['import', 'os', 'sys', 'open', 'eval', 'exec', '__', 'subprocess', 'pickle', 'request']
      if any(f in code for f in forbidden):
          return "خطا: عملیات غیرمجاز"

      try:
          # eval اول
          result = eval(code, {"__builtins__": {}}, self.allowed_globals)
          return result
      except Exception as e1:
          try:
              local = {}
              exec(code, {"__builtins__": {}}, {**self.allowed_globals, **local})
              return local.get('result', str(e1))
          except Exception as e2:
              return f"خطا: {str(e2)}"

In [ ]:
# bot
#from llm import PandasCodeGenerator
#from executor import SafeExecutor

# لود دیتاست
df = pd.read_csv(r"/data/df_final.csv") # مسیر دیتاست خود را قرار دهید
executor = SafeExecutor(df)
generator = PandasCodeGenerator()

# اطلاعات دیتافریم برای پرامپت
df_info = ", ".join(df.columns) + f" | Shape: {df.shape}"

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "سلام! هر سوالی درباره دیتاست بپرس.\n"
        "مثال: میانگین حقوق کارمندان زن چقدره؟"
    )

def escape_markdown_v2(text: str) -> str:
    """
    فرار کردن کاراکترهای خطرناک در MarkdownV2 تلگرام
    """
    escape_chars = r'\_*[]()~`>#+-=|{}.!'
    return re.sub(f'([{re.escape(escape_chars)}])', r'\\\1', text)

async def handle_question(update: Update, context: ContextTypes.DEFAULT_TYPE):
    question = update.message.text
    await update.message.reply_text("در حال پردازش...")

    try:
        code = generator.generate_pandas_code(question, df_info)
        result = executor.execute(code)

        # تبدیل نتیجه به متن
        if isinstance(result, (int, float)):
            result_str = f"{result:,.2f}"
        elif isinstance(result, pd.Series):
            result_str = result.head(10).to_string()
        elif isinstance(result, pd.DataFrame):
            result_str = result.head(5).to_string()
        else:
            result_str = str(result)

        # فرار کردن فقط متن و کد (نه نتیجه داخل ```)
        escaped_question = escape_markdown_v2(question)
        escaped_code = escape_markdown_v2(code)

        response = f"""
سوال: {escaped_question}

کد تولید شده:
`{escaped_code}`

نتیجه:
```{result_str}```
""".strip()

        await update.message.reply_markdown_v2(response)

    except Exception as e:
        error_msg = escape_markdown_v2(str(e))
        await update.message.reply_markdown_v2(f"خطا: {error_msg}")

In [ ]:
# main
#from bot import start, handle_question

nest_asyncio.apply()

TOKEN = "Your Telegram Bot Token Here" #توکن تلگرام خود را در این قسمت قرار دهید

async def main():
    app = Application.builder().token(TOKEN).build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_question))
    print("ربات در حال اجراست...")
    await app.run_polling()

if __name__ == "__main__":
    asyncio.run(main())